# SVD-Hybrid Task Merging Experiment - 8 Tasks on CLIP Models

This notebook runs SVD-Hybrid merging experiments on 8 fine-tuned CLIP models.

## Setup

**Google Drive Structure Expected:**
```
My Drive/
└── clip_finetune/
    ├── ViT-B-16/
    │   ├── Cars/finetuned.pt
    │   ├── DTD/finetuned.pt
    │   ├── EuroSAT/finetuned.pt
    │   ├── GTSRB/finetuned.pt
    │   ├── MNIST/finetuned.pt
    │   ├── RESISC45/finetuned.pt
    │   ├── SUN397/finetuned.pt
    │   ├── SVHN/finetuned.pt
    │   ├── Cars_head.pt
    │   ├── DTD_head.pt
    │   └── ... (head files for each dataset)
    ├── ViT-B-32/
    │   └── ... (same structure)
    └── ViT-L-14/
        └── ... (same structure)
```

## 1. Installation & Setup

In [ ]:
# Install required dependencies
!pip install torch torchvision numpy scikit-learn scipy open_clip_torch -q

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone the repository
!git clone https://github.com/mgradyn/SVD-Quantization-Task-Merging.git
%cd SVD-Quantization-Task-Merging

In [ ]:
import os
import sys
import torch
import json
from pathlib import Path
from typing import Dict, List, Optional

# Add repository to path
sys.path.insert(0, '/content/SVD-Quantization-Task-Merging')

# Import SVD-Hybrid components
from src.svd_hybrid.config import SVDHybridConfig
from src.svd_hybrid.cli import run_svd_hybrid_pipeline
from src.svd_hybrid.task_vector_loader import load_checkpoint, compute_task_vector
from task_vectors import TaskVector
from dataset_constants import STANDARD_8_TASKS, normalize_task_name

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuration

Configure paths and model settings. Adjust these based on your Google Drive setup.

In [ ]:
# =============================================================================
# CONFIGURATION - Modify these paths based on your Google Drive structure
# =============================================================================

# Base path to checkpoints in Google Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/clip_finetune"

# Available models (choose one)
AVAILABLE_MODELS = ["ViT-B-16", "ViT-B-32", "ViT-L-14"]

# Select model to use for this experiment
SELECTED_MODEL = "ViT-B-32"  # Change to ViT-B-16 or ViT-L-14 as needed

# The 8 standard evaluation tasks
TASKS = ["Cars", "DTD", "EuroSAT", "GTSRB", "MNIST", "RESISC45", "SUN397", "SVHN"]

# Output directory (in Colab's local storage)
OUTPUT_DIR = "/content/output"

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Selected model: {SELECTED_MODEL}")
print(f"Tasks: {TASKS}")
print(f"Device: {DEVICE}")

In [ ]:
# =============================================================================
# PATH SETUP - Construct paths based on Google Drive structure
# =============================================================================

def get_model_paths(base_path: str, model_name: str, tasks: List[str]) -> dict:
    """
    Get all checkpoint and head paths for a model.
    
    Expected structure:
    - clip_finetune/{model_name}/{task}/finetuned.pt
    - clip_finetune/{model_name}/{task}_head.pt (or similar naming)
    """
    model_dir = os.path.join(base_path, model_name)
    
    paths = {
        "model_dir": model_dir,
        "checkpoints": {},
        "heads": {},
    }
    
    for task in tasks:
        # Finetuned checkpoint path
        checkpoint_path = os.path.join(model_dir, task, "finetuned.pt")
        paths["checkpoints"][task] = checkpoint_path
        
        # Head path (try different naming conventions)
        possible_head_paths = [
            os.path.join(model_dir, f"{task}_head.pt"),
            os.path.join(model_dir, f"{task.lower()}_head.pt"),
            os.path.join(model_dir, f"head_{task}.pt"),
            os.path.join(model_dir, task, "head.pt"),
        ]
        
        for head_path in possible_head_paths:
            if os.path.exists(head_path):
                paths["heads"][task] = head_path
                break
    
    return paths

# Get paths for selected model
model_paths = get_model_paths(DRIVE_BASE_PATH, SELECTED_MODEL, TASKS)

print(f"\nModel directory: {model_paths['model_dir']}")
print(f"\nCheckpoint paths:")
for task, path in model_paths["checkpoints"].items():
    exists = "✓" if os.path.exists(path) else "✗"
    print(f"  {exists} {task}: {path}")

In [ ]:
# =============================================================================
# VERIFY ALL PATHS EXIST
# =============================================================================

def verify_paths(model_paths: dict) -> bool:
    """
    Verify all required paths exist.
    """
    all_valid = True
    
    print("Verifying checkpoint paths...")
    for task, path in model_paths["checkpoints"].items():
        if os.path.exists(path):
            size_mb = os.path.getsize(path) / (1024 * 1024)
            print(f"  ✓ {task}: {size_mb:.1f} MB")
        else:
            print(f"  ✗ {task}: NOT FOUND at {path}")
            all_valid = False
    
    print("\nVerifying head paths...")
    for task, path in model_paths.get("heads", {}).items():
        if os.path.exists(path):
            size_mb = os.path.getsize(path) / (1024 * 1024)
            print(f"  ✓ {task} head: {size_mb:.2f} MB")
        else:
            print(f"  ✗ {task} head: NOT FOUND")
    
    # Check for any head files in the model directory
    if os.path.exists(model_paths["model_dir"]):
        print("\nAll .pt files in model directory:")
        for f in sorted(os.listdir(model_paths["model_dir"])):
            if f.endswith(".pt"):
                full_path = os.path.join(model_paths["model_dir"], f)
                size_mb = os.path.getsize(full_path) / (1024 * 1024)
                print(f"  - {f}: {size_mb:.2f} MB")
    
    return all_valid

paths_valid = verify_paths(model_paths)

if not paths_valid:
    print("\n⚠️ Some paths are missing. Please verify your Google Drive structure.")
else:
    print("\n✓ All paths verified successfully!")

## 3. Load Base Model

We need the base (pretrained) CLIP model to compute task vectors.

In [ ]:
import open_clip

def get_clip_model_name(model_name: str) -> tuple:
    """
    Convert model name to open_clip format.
    """
    model_map = {
        "ViT-B-16": ("ViT-B-16", "openai"),
        "ViT-B-32": ("ViT-B-32", "openai"),
        "ViT-L-14": ("ViT-L-14", "openai"),
    }
    return model_map.get(model_name, (model_name, "openai"))

def load_base_clip_model(model_name: str, device: str = "cpu"):
    """
    Load the base (pretrained) CLIP model.
    """
    clip_name, pretrained = get_clip_model_name(model_name)
    print(f"Loading base model: {clip_name} ({pretrained})...")
    
    model, _, preprocess = open_clip.create_model_and_transforms(
        clip_name, 
        pretrained=pretrained,
        device=device
    )
    
    # Get the state dict (visual encoder only for task vectors)
    base_state_dict = model.visual.state_dict()
    
    print(f"Loaded base model with {len(base_state_dict)} parameters")
    return model, base_state_dict, preprocess

# Load base model
base_model, base_state_dict, preprocess = load_base_clip_model(SELECTED_MODEL, DEVICE)

# Save base state dict for task vector computation
BASE_MODEL_PATH = "/content/base_model.pt"
torch.save(base_state_dict, BASE_MODEL_PATH)
print(f"\nBase model saved to: {BASE_MODEL_PATH}")

## 4. Load Task Vectors

Compute task vectors (delta = finetuned - base) for each task.

In [ ]:
def load_task_vectors_from_checkpoints(
    base_state_dict: Dict[str, torch.Tensor],
    checkpoint_paths: Dict[str, str],
    device: str = "cpu"
) -> Dict[str, Dict[str, torch.Tensor]]:
    """
    Load task vectors from finetuned checkpoints.
    
    Args:
        base_state_dict: Base model state dict
        checkpoint_paths: Dict mapping task names to checkpoint paths
        device: Device to load to
    
    Returns:
        Dict mapping task names to task vectors (delta dicts)
    """
    task_vectors = {}
    
    for task, path in checkpoint_paths.items():
        if not os.path.exists(path):
            print(f"  ⚠️ Skipping {task}: checkpoint not found")
            continue
        
        print(f"  Loading {task}...")
        
        try:
            # Load finetuned checkpoint
            finetuned_state = torch.load(path, map_location=device)
            
            # Handle nested state dicts
            if isinstance(finetuned_state, dict):
                if "state_dict" in finetuned_state:
                    finetuned_state = finetuned_state["state_dict"]
                elif "model" in finetuned_state:
                    finetuned_state = finetuned_state["model"]
                elif "visual" in finetuned_state:
                    finetuned_state = finetuned_state["visual"]
            
            # Compute task vector
            task_vector = compute_task_vector(base_state_dict, finetuned_state, device)
            
            if len(task_vector) > 0:
                task_vectors[task] = task_vector
                print(f"    ✓ {len(task_vector)} parameters")
            else:
                print(f"    ⚠️ No matching parameters found")
                
        except Exception as e:
            print(f"    ✗ Error: {e}")
    
    return task_vectors

print("Loading task vectors...\n")
task_vectors = load_task_vectors_from_checkpoints(
    base_state_dict, 
    model_paths["checkpoints"], 
    DEVICE
)

print(f"\nLoaded {len(task_vectors)} task vectors")

In [ ]:
# =============================================================================
# ANALYZE TASK VECTORS
# =============================================================================

def analyze_task_vectors(task_vectors: Dict[str, Dict[str, torch.Tensor]]):
    """
    Analyze task vectors to understand their characteristics.
    """
    print("Task Vector Analysis")
    print("=" * 60)
    
    for task_name, task_vector in task_vectors.items():
        total_params = sum(t.numel() for t in task_vector.values())
        total_size_mb = sum(t.numel() * t.element_size() for t in task_vector.values()) / (1024 * 1024)
        
        # Compute statistics
        all_values = torch.cat([t.flatten() for t in task_vector.values()])
        mean_val = all_values.mean().item()
        std_val = all_values.std().item()
        max_val = all_values.abs().max().item()
        
        print(f"\n{task_name}:")
        print(f"  Parameters: {len(task_vector)}")
        print(f"  Total elements: {total_params:,}")
        print(f"  Size: {total_size_mb:.2f} MB")
        print(f"  Mean: {mean_val:.6f}")
        print(f"  Std: {std_val:.6f}")
        print(f"  Max abs: {max_val:.6f}")

if task_vectors:
    analyze_task_vectors(task_vectors)

## 5. Load Classification Heads

Load the task-specific classification heads for evaluation.

In [ ]:
def find_and_load_heads(model_dir: str, tasks: List[str], device: str = "cpu"):
    """
    Find and load classification heads for all tasks.
    """
    heads = {}
    
    if not os.path.exists(model_dir):
        print(f"Model directory not found: {model_dir}")
        return heads
    
    # List all .pt files in model directory
    pt_files = [f for f in os.listdir(model_dir) if f.endswith(".pt")]
    
    print("Available .pt files:")
    for f in sorted(pt_files):
        print(f"  - {f}")
    
    print("\nLoading heads...")
    for task in tasks:
        # Try different naming patterns
        possible_names = [
            f"{task}_head.pt",
            f"{task.lower()}_head.pt",
            f"head_{task}.pt",
            f"head_{task.lower()}.pt",
            f"{task}Head.pt",
            f"{task}_classifier.pt",
        ]
        
        head_loaded = False
        for name in possible_names:
            path = os.path.join(model_dir, name)
            if os.path.exists(path):
                try:
                    head = torch.load(path, map_location=device)
                    heads[task] = head
                    print(f"  ✓ {task}: {name}")
                    head_loaded = True
                    break
                except Exception as e:
                    print(f"  ✗ {task}: Error loading {name}: {e}")
        
        if not head_loaded:
            print(f"  ⚠️ {task}: No head found")
    
    return heads

# Load heads
heads = find_and_load_heads(model_paths["model_dir"], TASKS, DEVICE)
print(f"\nLoaded {len(heads)} classification heads")

## 6. Run SVD-Hybrid Merging Experiment

Now we run the SVD-Hybrid merging with different configurations.

In [ ]:
# =============================================================================
# SETUP CHECKPOINT DIRECTORY FOR SVD-HYBRID
# =============================================================================

# Create a checkpoint directory with the expected structure
CHECKPOINT_DIR = "/content/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Create symlinks or copy checkpoints to expected locations
for task in TASKS:
    source = model_paths["checkpoints"][task]
    task_dir = os.path.join(CHECKPOINT_DIR, task)
    os.makedirs(task_dir, exist_ok=True)
    
    dest = os.path.join(task_dir, "finetuned.pt")
    
    if os.path.exists(source) and not os.path.exists(dest):
        # Create symlink to avoid copying large files
        os.symlink(source, dest)
        print(f"✓ Linked {task}")
    elif os.path.exists(dest):
        print(f"✓ {task} already linked")
    else:
        print(f"✗ {task} source not found")

print(f"\nCheckpoint directory: {CHECKPOINT_DIR}")

In [ ]:
# =============================================================================
# RUN SVD-HYBRID MERGING
# =============================================================================

def run_experiment(
    tasks: List[str],
    checkpoint_dir: str,
    base_model_path: str,
    output_dir: str,
    config_name: str = "default",
    energy_threshold: float = 0.95,
    max_rank: int = 64,
    weighting: str = "uniform",
    device: str = "cuda"
):
    """
    Run SVD-Hybrid merging experiment with given configuration.
    """
    print(f"\n{'='*60}")
    print(f"Running experiment: {config_name}")
    print(f"{'='*60}")
    print(f"  Energy threshold: {energy_threshold}")
    print(f"  Max rank: {max_rank}")
    print(f"  Weighting: {weighting}")
    print(f"  Tasks: {tasks}")
    
    # Create output directory for this experiment
    exp_output_dir = os.path.join(output_dir, config_name)
    os.makedirs(exp_output_dir, exist_ok=True)
    
    # Create configuration
    config = SVDHybridConfig(
        tasks=tasks,
        model=SELECTED_MODEL,
        checkpoint_dir=checkpoint_dir,
        base_model_path=base_model_path,
        mask_dir="",  # No masks for this experiment
        svd_energy_threshold=energy_threshold,
        svd_max_rank=max_rank,
        svd_center=True,
        svd_fp16=True,
        svd_low_bits=4,
        svd_rtvq_stages=2,
        svd_weighting=weighting,
        svd_store_artifacts=True,
        svd_eval_reconstruction=True,
        output_dir=exp_output_dir,
        artifact_dir=os.path.join(exp_output_dir, "artifacts"),
        device=device
    )
    
    # Run pipeline
    try:
        results = run_svd_hybrid_pipeline(config)
        print(f"\n✓ Experiment '{config_name}' completed successfully!")
        print(f"  Output saved to: {exp_output_dir}")
        return results
    except Exception as e:
        print(f"\n✗ Experiment '{config_name}' failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Filter tasks to only those with valid checkpoints
valid_tasks = [t for t in TASKS if t in task_vectors]
print(f"Valid tasks with checkpoints: {valid_tasks}")

# Run default experiment
if len(valid_tasks) >= 2:
    results_default = run_experiment(
        tasks=valid_tasks,
        checkpoint_dir=CHECKPOINT_DIR,
        base_model_path=BASE_MODEL_PATH,
        output_dir=OUTPUT_DIR,
        config_name="uniform_e95_r64",
        energy_threshold=0.95,
        max_rank=64,
        weighting="uniform",
        device=DEVICE
    )
else:
    print("⚠️ Not enough valid tasks to run experiment. Need at least 2 tasks.")

In [ ]:
# =============================================================================
# RUN EXPERIMENT SWEEP (OPTIONAL)
# =============================================================================

RUN_SWEEP = False  # Set to True to run multiple configurations (resource intensive)

if RUN_SWEEP and len(valid_tasks) >= 2:
    # Define experiment configurations
    experiments = [
        {"name": "uniform_e90_r32", "energy": 0.90, "rank": 32, "weight": "uniform"},
        {"name": "uniform_e95_r64", "energy": 0.95, "rank": 64, "weight": "uniform"},
        {"name": "uniform_e99_r128", "energy": 0.99, "rank": 128, "weight": "uniform"},
    ]
    
    all_results = {}
    
    for exp in experiments:
        results = run_experiment(
            tasks=valid_tasks,
            checkpoint_dir=CHECKPOINT_DIR,
            base_model_path=BASE_MODEL_PATH,
            output_dir=OUTPUT_DIR,
            config_name=exp["name"],
            energy_threshold=exp["energy"],
            max_rank=exp["rank"],
            weighting=exp["weight"],
            device=DEVICE
        )
        
        if results is not None:
            all_results[exp["name"]] = results
    
    print(f"\n{'='*60}")
    print(f"Completed {len(all_results)}/{len(experiments)} experiments")
    print(f"{'='*60}")
else:
    print("Skipping experiment sweep")

## 7. Simple Task Arithmetic Baseline (Optional)

For comparison, we also perform simple task arithmetic (averaging task vectors).

In [ ]:
# =============================================================================
# SIMPLE TASK ARITHMETIC BASELINE
# =============================================================================

def simple_task_arithmetic(
    base_state_dict: Dict[str, torch.Tensor],
    task_vectors: Dict[str, Dict[str, torch.Tensor]],
    scaling_factor: float = 1.0
) -> Dict[str, torch.Tensor]:
    """
    Simple task arithmetic: average task vectors and add to base model.
    
    merged = base + scaling_factor * mean(task_vectors)
    """
    # Get all parameter names
    all_params = set()
    for tv in task_vectors.values():
        all_params.update(tv.keys())
    
    # Average task vectors
    merged_state = {}
    n_tasks = len(task_vectors)
    
    for param_name in base_state_dict.keys():
        base_param = base_state_dict[param_name]
        
        if param_name in all_params:
            # Compute average delta
            deltas = [tv[param_name] for tv in task_vectors.values() if param_name in tv]
            avg_delta = sum(deltas) / len(deltas)
            
            # Apply to base
            merged_state[param_name] = base_param + scaling_factor * avg_delta
        else:
            # Keep base value
            merged_state[param_name] = base_param
    
    return merged_state

# Run simple task arithmetic
if task_vectors:
    print("Running simple task arithmetic baseline...")
    
    for scaling in [0.3, 0.5, 1.0]:
        merged_simple = simple_task_arithmetic(
            base_state_dict, 
            task_vectors, 
            scaling_factor=scaling
        )
        
        # Save merged model
        simple_output_path = os.path.join(OUTPUT_DIR, f"simple_merge_scale{scaling}.pt")
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        torch.save(merged_simple, simple_output_path)
        print(f"  ✓ Saved simple merge (scale={scaling}) to: {simple_output_path}")
else:
    print("⚠️ No task vectors loaded, skipping simple merge")

## 8. View Results

In [ ]:
# =============================================================================
# VIEW EXPERIMENT RESULTS
# =============================================================================

def load_diagnostics(output_dir: str, config_name: str) -> Optional[dict]:
    """
    Load diagnostics from an experiment.
    """
    diag_path = os.path.join(output_dir, config_name, "artifacts", "diagnostics.json")
    
    if os.path.exists(diag_path):
        with open(diag_path, 'r') as f:
            return json.load(f)
    
    # Try alternate path
    diag_path = os.path.join(output_dir, config_name, "diagnostics.json")
    if os.path.exists(diag_path):
        with open(diag_path, 'r') as f:
            return json.load(f)
    
    return None

def print_experiment_summary(output_dir: str):
    """
    Print summary of all experiments.
    """
    print("\n" + "="*60)
    print("EXPERIMENT RESULTS SUMMARY")
    print("="*60)
    
    if not os.path.exists(output_dir):
        print(f"Output directory not found: {output_dir}")
        return
    
    for exp_name in sorted(os.listdir(output_dir)):
        exp_path = os.path.join(output_dir, exp_name)
        
        if not os.path.isdir(exp_path):
            continue
        
        print(f"\n{exp_name}:")
        
        # Load diagnostics
        diagnostics = load_diagnostics(output_dir, exp_name)
        
        if diagnostics and "summary" in diagnostics:
            summary = diagnostics["summary"]
            print(f"  Num parameters: {summary.get('num_parameters', 'N/A')}")
            print(f"  Avg rank: {summary.get('average_rank', 'N/A'):.1f}")
            print(f"  Avg energy retained: {summary.get('average_energy_retained', 'N/A'):.4f}")
            print(f"  Avg reconstruction error: {summary.get('average_reconstruction_error', 'N/A'):.6f}")
        
        # Check if merged model exists
        merged_path = os.path.join(exp_path, "merged_state_dict.pt")
        if os.path.exists(merged_path):
            size_mb = os.path.getsize(merged_path) / (1024 * 1024)
            print(f"  Merged model size: {size_mb:.2f} MB")

print_experiment_summary(OUTPUT_DIR)

In [ ]:
# =============================================================================
# LIST ALL OUTPUT FILES
# =============================================================================

def list_directory_tree(path: str, prefix: str = "", max_depth: int = 3, current_depth: int = 0):
    """
    Print directory tree.
    """
    if current_depth > max_depth:
        return
    
    if not os.path.exists(path):
        return
    
    items = sorted(os.listdir(path))
    
    for i, item in enumerate(items):
        is_last = (i == len(items) - 1)
        connector = "└── " if is_last else "├── "
        
        item_path = os.path.join(path, item)
        
        if os.path.isdir(item_path):
            print(f"{prefix}{connector}{item}/")
            extension = "    " if is_last else "│   "
            list_directory_tree(item_path, prefix + extension, max_depth, current_depth + 1)
        else:
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"{prefix}{connector}{item} ({size_mb:.2f} MB)")

print("\nOutput Directory Structure:")
print("="*60)
if os.path.exists(OUTPUT_DIR):
    list_directory_tree(OUTPUT_DIR)
else:
    print(f"Output directory not found: {OUTPUT_DIR}")

## 9. Save Results to Google Drive

In [ ]:
# =============================================================================
# SAVE RESULTS TO GOOGLE DRIVE
# =============================================================================

SAVE_TO_DRIVE = True  # Set to True to save results to Google Drive
DRIVE_OUTPUT_DIR = f"/content/drive/MyDrive/svd_hybrid_results/{SELECTED_MODEL}"

if SAVE_TO_DRIVE and os.path.exists(OUTPUT_DIR):
    import shutil
    
    # Create output directory in Google Drive
    os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
    
    # Copy results
    print(f"Saving results to Google Drive: {DRIVE_OUTPUT_DIR}")
    
    for item in os.listdir(OUTPUT_DIR):
        src = os.path.join(OUTPUT_DIR, item)
        dst = os.path.join(DRIVE_OUTPUT_DIR, item)
        
        if os.path.isdir(src):
            if os.path.exists(dst):
                shutil.rmtree(dst)
            shutil.copytree(src, dst)
            print(f"  ✓ Copied {item}/")
        else:
            shutil.copy2(src, dst)
            print(f"  ✓ Copied {item}")
    
    print(f"\n✓ Results saved to: {DRIVE_OUTPUT_DIR}")
else:
    print("Skipping save to Google Drive")

## 10. Cleanup (Optional)

In [ ]:
# =============================================================================
# CLEANUP (OPTIONAL)
# =============================================================================

CLEANUP = False  # Set to True to clean up temporary files

if CLEANUP:
    import shutil
    
    # Remove temporary directories
    temp_dirs = [
        CHECKPOINT_DIR,
        OUTPUT_DIR,
        "/content/SVD-Quantization-Task-Merging",
    ]
    
    for d in temp_dirs:
        if os.path.exists(d):
            shutil.rmtree(d)
            print(f"Removed: {d}")
    
    # Remove base model
    if os.path.exists(BASE_MODEL_PATH):
        os.remove(BASE_MODEL_PATH)
        print(f"Removed: {BASE_MODEL_PATH}")
    
    print("\n✓ Cleanup complete")
else:
    print("Cleanup skipped. Set CLEANUP = True to remove temporary files.")

## Summary

This notebook performed the following:

1. **Setup**: Installed dependencies and mounted Google Drive
2. **Configuration**: Set up paths for checkpoints in Google Drive
3. **Base Model**: Loaded pretrained CLIP model
4. **Task Vectors**: Computed task vectors from fine-tuned checkpoints
5. **Heads**: Loaded classification heads for evaluation
6. **SVD-Hybrid Merging**: Ran experiments with different configurations
7. **Simple Baseline**: Performed simple task arithmetic for comparison
8. **Results**: Displayed experiment results and diagnostics
9. **Save**: Optionally saved results to Google Drive

### Next Steps

- Evaluate merged models on test datasets
- Compare SVD-Hybrid results with baseline methods
- Tune hyperparameters (energy threshold, max rank, etc.)
- Try different weighting strategies (performance-based, cluster-based)